## Imports and setting up GDAL environment variables

In [ ]:
import json
import datetime
import requests as rq

import numpy as np

from PIL import Image

import folium
from folium import plugins

from osgeo import gdal

import shapely
import shapely.wkt
import shapely.geometry
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform as shapely_transform

import pyproj
from pyproj import Proj, transform

import rasterio as rio
from rasterio.mask import mask


gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', '~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'YES')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'TIF')

# Functions used throughout notebook

- **reproject_aoi** : function takes CRS.from_espg and a WGS Shapely Polygon, to reproject WGS Shapely polygon to desired CRS
- **get_geojson_from_poly** : getting GeoJSON in list (as requried for rasterio mask) from any Shapely Polygon
- **add_geojson_to_map** : Adding a GeoJSON to Folium map
- **reproject2wgs_write** : function that takes a raster and reprojects tp WGS84 and writes output as geotiff
- **add_array_to_map** : function takes raster array, its bounds, and adds it to Folium basemap as Overlay
- **get_wgs_coords** : function that takes lat long pair in any CRS to give output in WGS84 CRS

In [ ]:
def reproject_aoi(crs, aoishape):
    wgs84 = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
    utm = pyproj.Proj(crs)
    trans_utm = pyproj.Transformer.from_proj(wgs84, utm)
    utm_shapely_polygon = shapely_transform(trans_utm.transform, aoishape)

    return utm_shapely_polygon

def get_geojson_from_poly(shapely_polygon):
    wktext = shapely.wkt.loads(str(shapely_polygon))
    json_str = json.dumps(shapely.geometry.mapping(wktext))

    geometry = [json.loads(json_str)]
    return geometry

def add_geojson_to_map(geojson, map):
    folium.GeoJson(geojson['geometry'],
        name = geojson['properties']['id'],
        zoom_on_click=True,
        style_function=styles).add_to(map)

def add_array_to_map(array,bounds,name):
    folium.raster_layers.ImageOverlay(
        image=array,
        name=name,
        opacity=1,
        bounds= boundary
    ).add_to(m)

def get_wgs_coords(lat,lon,crs):

    inProj = Proj(crs)
    outProj = Proj('epsg:4326')
    x1,y1 = lon,lat
    x2,y2 = transform(inProj,outProj,x1,y1)
    return x2,y2

### Using GET request to download raw Geojson from github gist link

In [ ]:
gist = rq.get('https://gist.githubusercontent.com/thaisbendixen/e126c37a3fa021495414658eeaf86d8d/raw/5d1926dcb3a4b9d631521ba12ea79fdc1ecd2df7/doberitz_multipolygon.geojson')
geoj = gist.json()

#### Naming polygons within Multipolygon for use later

In [ ]:
## Adding "poly_ID" to the two polygons in the multipolygon geojson for later use

for i in range(len(geoj['features'])):
    geoj['features'][i]['properties']['id'] = f"poly_{i+1}"

In [ ]:
geoj

### Creating Multipolygon in Shapely for geometric operations on the given geojson

In [ ]:
multishape = MultiPolygon([shape(pol['geometry']) for pol in geoj['features']])

### Setting up Folium Basemap layers and GeoJSON styles

In [ ]:
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}


styles =lambda feature: {
        "fillColor": "blue",
        "color": "white",
        "weight": 2,
        "dashArray": "5, 5"}


### Visualizing the given GeoJSON on Folium

In [ ]:
m = folium.Map([multishape.centroid.y,multishape.centroid.x], zoom_start=12)

# Add custom basemaps
basemaps['Google Maps'].add_to(m)
basemaps['Google Satellite Hybrid'].add_to(m)

for feature in geoj['features']:
    add_geojson_to_map(feature,m)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

#fullscreen
plugins.Fullscreen().add_to(m)
    
m

## Here, we user the JSON response output from the Python CLI script to Visualize Sentinel-2 RGB image and KMeans output image on Folium

In [ ]:
import json

with open('./analysis_response.json') as d:
    data = json.load(d)


In [ ]:
data

### Note - 
- The colors in the PNG image to be overlain on the Folium basemap is generated using the "gdaldem" cli function and the associated "colormap.txt" file.

- The outputs will be colored differently on new runs of the KMeans, although "random_state" of KMeans was set to 0.

#### Hence a prepared output in Matplotlib is attached at the bottom of the notebook, with proper colors and legend.

-------------

In [ ]:
m = folium.Map(location=[multishape.centroid.y,multishape.centroid.x], tiles='Stamen Terrain', zoom_start=10)

for info in data:

    print(info['polygon_id'])
    
    # opening the Kmeans output of the polygon to get CRS and Bounds information
    out_dat = rio.open(f"./kmeans_output/Unsupervised_KMeans_{info['polygon_id']}.tif")
    
    utmbounds = out_dat.bounds
    
    # converting UTM bounds to WGS bounds, for use in Folium Overlay
    bot, left = get_wgs_coords(utmbounds.bottom, utmbounds.left, out_dat.crs.to_dict()['init'])
    top, right = get_wgs_coords(utmbounds.top, utmbounds.right, out_dat.crs.to_dict()['init'])
    
    # getting True Color Image associated with polygon
    tci = rio.open(info['true_color_image'])
    
    # getting the geometry of the polygon, to clip true color image
    for i in range(len(geoj['features'])):
        if info['polygon_id'] == geoj['features'][i]['properties']['id']:
            geom = geoj['features'][i]['geometry']
    
    #reprojcting to UTM zome of sen2 image
    utm_shape = reproject_aoi(out_dat.crs, shape(geom))
    
    # clipping COG
    tci_array, tci_transf = mask(tci,get_geojson_from_poly(utm_shape),crop=True)
    
    # transposing Numpy array for Folium
    tcirgb = tci_array.transpose(1,-1,0)

    boundary = [[bot, left],[top,right]]
    
    # adding True Color Image to Folium as Overlay
    add_array_to_map(tcirgb, boundary, f"true_color_image_{info['polygon_id']}")
    
    # adding the PNG output created by Python CLI script, as Folium Overlay
    png_array = np.array(Image.open(f"./kmeans_output/Unsupervised_KMeans_{info['polygon_id']}.png"))
    add_array_to_map(png_array, boundary, f"KMeans_output_{info['polygon_id']}")

# adding extra basemaps to Folium
basemaps['Google Maps'].add_to(m)
basemaps['Google Satellite Hybrid'].add_to(m)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

m

# Conclusions -

### The KMeans clusters Identify 4 different clusters in both areas of Interest.

### A prepared Matplotlib PNG output can be seen below, explain the land covers that are present and identified by the KMeans Unsupervised Algorithm

![](./png_outputs/Unsupervised_poly_1.png) ![](./png_outputs/Unsupervised_poly_2.png)